In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(151)

# 演習1 形態素解析

## データのロード

In [ ]:
import json
import gzip

In [ ]:
with gzip.open("data/jawikinews-20170201.json.gz") as fp:
    data = json.loads(fp.read().decode("utf-8"))

## 正規化

タイトル部分: 全角英数を半角に変換します。
本文部分: 全角英数を半角に変換します。また、句点単位で文に分割します。ただし、カギカッコ内や「モーニング娘。」などの句点は無視します。

In [ ]:
import unicodedata

In [ ]:
normalized = {}

In [ ]:
titles = []
for title in data["title"]:
    # 全角英数を半角英数に変換
    titles.append(unicodedata.normalize("NFKC", title))
normalized["title"] = titles

In [ ]:
def split_line_sub(text):
    count = 0
    line = ""
    # 1文字ずつ確認
    for c in text:
        # 括弧内にいるときはcount > 0になるようにする。
        if c in ("「", "『"):
            count += 1
        elif c in ("」", "』"):
            count -= 1
            if count < 0:
                count = 0
        line += c
        # 括弧の外 かつ 句点が現れたら改行文字を挿入
        if c == "。" and count == 0:
            line += "\n"
    return line

def split_line(text):
    """
    文書を文のリストに変換
    """
    # 一旦改行文字やタブをスペースに変換
    text = text.replace("\n", " ").replace("\t", " ")
    # モーニング娘。で句点分割されないように、別の文字列に変換しておく
    text = text.replace(
        "モーニング娘。", "<MorningMusume>").replace("モー娘。", "<MorMusu>")
    # 句点で切り分ける
    text = split_line_sub(text)
    
    text = # TODO モーニング娘。を元に戻す
    
    # 文書(文字列) を 文のリストに変換
    lines = text.split("\n")
    # 文の前後にスペースがあったら削除
    lines = map(lambda x: x.strip(), lines)
    # 空文字は削除
    return list(filter(lambda x: x != "", lines))

In [ ]:
texts = []
for text in data["text"]:
    # 全角英数を半角英数に変換
    text = unicodedata.normalize("NFKC", text)
    lines = split_line(text)
    texts.append(lines)
normalized["text"] = texts

## 形態素解析

In [ ]:
from janome.tokenizer import Tokenizer

In [ ]:
tagger = Tokenizer()

In [ ]:
def parse_orig(text):
    """
    文を形態素解析し、単語のリストに変換する。
    ただし、単語は原型に戻す。
    """
    words = []
    for token in : # TODO textを形態素解析
        orig_form = token.base_form
        if orig_form == "*":
            # 特殊単語(数字など)の場合、表層(実際に書かれている文字列)を原型とみなす
            orig_form = token.surface
            
        orig_form = # TODO 原型から前後のスペースを除去し、アルファベットは小文字に変換
        if orig_form != "":
            words.append(orig_form)
    return words

In [ ]:
titles = []
for title in normalized["title"]:
    tokenized = parse_orig(title)
    titles.append(tokenized)

In [ ]:
texts = []
for lines in normalized["text"]:
    doc = []
    for line in lines:
        tokenized = parse_orig(line)
        doc.append(tokenized)
    texts.append(doc)

In [ ]:
texts[0][0]

# 演習2 TF-IDF

TF-IDFを使って類似記事をレコメンドしてみましょう。

TF-IDF計算用に、scikit learnが認識する形にデータを変形します。

In [ ]:
texts_for_tfidf = []
for doc in texts:
    lines = []
    for words in doc:
        lines.append(" ".join(words))
    texts_for_tfidf.append("\n".join(lines))

In [ ]:
texts_for_tfidf[0]

### ストップワードを作りましょう

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

各記事の単語をカウントしましょう。

In [ ]:
count_vectorizer = # TODO CountVectorizerを使ってカウント用オブジェクトを作ってください

In [ ]:
counts = count_vectorizer.fit_transform(texts_for_tfidf)

文書数x単語数の行列ができました

In [ ]:
counts.shape

どの位置にどの単語が対応するかはvocabulary_を見るとわかります。

In [ ]:
list(count_vectorizer.vocabulary_.items())[:10]

コーパス全体における各単語の出現回数を数えましょう。

In [ ]:
freq = np.array(counts.sum(axis=0))[0]

In [ ]:
freq

今回は出現回数上位75件をストップワードにしましょう。

In [ ]:
stopword_ids = freq.argsort()[-75:]

In [ ]:
stopwords = []
for word_id in stopword_ids:
    for word, idx in count_vectorizer.vocabulary_.items():
        if word_id == idx:
            stopwords.append(word)

In [ ]:
print(stopwords)

## TF-IDFを計算しましょう

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

TF-IDFを計算します。

In [ ]:
tfidf_vectorizer = # TfidfVectorizerを使ってTF-IDF計算用オブジェクトを作ってください。stopwordを指定するのを忘れないようにしましょう。

In [ ]:
tfidf = tfidf_vectorizer.fit_transform(texts_for_tfidf)

In [ ]:
tfidf.shape

In [ ]:
from scipy.sparse import lil_matrix

TF-IDFのトップn(=10)件を求め、記事をベクトルに変換します。

In [ ]:
def tfidf_to_bow(tfidf_matrix, n=10):
    # 結果格納用変数
    bow_matrix = lil_matrix(tfidf_matrix.shape, dtype=np.float)
    
    for i, doc in enumerate(tfidf_matrix):
        tfidf_doc = np.array(doc.todense())[0]
        pairs = []
        for word_id, val in enumerate(tfidf_doc):
            if val > 0:
                # 出現した単語のみを選択
                pairs.append((word_id, val))
                
        # TF-IDFの大きい順に並べ替え、トップn件を取り出す
        top_n_words = sorted(pairs, key=lambda x: x[1], reverse=True)[:n]
        
        # トップn件の単語位置に1を立てます。
        # 1行につきn件、1が立っていることになります。
        for word_id, _ in top_n_words:
            bow_matrix[i, word_id] = 1
            
    # CSRマトリックス(行計算が速い疎行列)に変換して返します。
    return bow_matrix.tocsr()

In [ ]:
bow = tfidf_to_bow(tfidf)

In [ ]:
bow.shape

各記事間の距離を計算します。コサイン距離を使います。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

記事の全組み合わせでコサイン距離を計算します。完全一致すると1になります。

In [ ]:
bow_sim = cosine_similarity(bow)

In [ ]:
bow_sim.shape

小さい順に並べ替え、2番目に値が大きい記事IDを取得します。(もっとも大きいのは自分自身との距離=完全一致=1なので、2番目をとります。)  
なお、argsortを使うと、記事の位置が取得できます。

In [ ]:
most_similar = np.argsort(bow_sim, axis=1)[:, -2]

In [ ]:
most_similar

先頭10件の類似記事を表示してみましょう。

In [ ]:
for i, sim in enumerate(most_similar):
    print("**************************************")
    print(data["text"][i])
    print("----")
    print(data["text"][sim])
    if i > 10:
        break

# 演習3 Word2Vec

In [ ]:
from gensim.models.word2vec import Word2Vec

Word2Vec学習用に、Word2Vecが認識できる形にデータを成形します。

In [ ]:
sentences = []
for doc in texts:
    sentences += doc

sentences += titles

Word2Vecのモデルを学習します。

In [ ]:
word2vec = Word2Vec(sentences)
word2vec.init_sims(replace=True)

(遊んでみましょう)

In [ ]:
word2vec.most_similar(positive=["東京", "西日本"], negative=["東日本"])

学習した単語ベクトルを用いて、文書のベクトルを計算します。

In [ ]:
def get_min_val(text):
    """
    文書内のベクトル要素の最小値を求めます。
    """
    min_val = 0
    for words in text:
        for word in words:
            if word in word2vec:
                m = word2vec[word].min()
                if min_val > m:
                    min_val = m
    return min_val
                
def text_to_vec(text, min_val):
    """
    文書をベクトルに変換します。
    """
    # 結果格納用変数
    vec = np.zeros(word2vec.layer1_size, dtype=np.float)
    
    # 総単語数カウント用変数
    n_words = 0
    
    for words in text:
        for word in words:
            if word in word2vec:
                # 単語ベクトルの要素が全て正の数になるよう、データを移動します。
                shift = word2vec[word] - min_val + 0.1
                
                # ベクトルの大きさを1に揃えます。
                norm = np.linalg.norm(shift)
                shift /= norm
                # ベクトルを対数変換し、結果格納用変数に加算します。
                # こうすることで、対数変換前の世界ではベクトルの要素積を行なったのと同値になります。
                vec += np.log(shift)
                n_words += 1
                
    if n_words > 0:
        # 総単語数で割り算することで、対数変換前の世界ではn_words乗根を計算したのと同値になります。
        # そこにexpを計算することで、対数変換前の世界に戻します。
        # データの移動を元に戻します。
        return np.exp(vec / n_words) + min_val - 0.1
    else:
        return vec

In [ ]:
vecs = []
min_val = 0
for text in texts:
    min_val = min(min_val, get_min_val(text))
    
for text in texts:
    vecs.append(text_to_vec(text, min_val))
vecs = np.array(vecs)

In [ ]:
vecs.shape

In [ ]:
vecs

文書の全組み合わせについてコサイン距離を計算します。

In [ ]:
vec_sim = # TODO 演習2を参考に、コサイン距離を計算してください。

In [ ]:
vec_sim.shape

もっとも類似した文書を求めます。

In [ ]:
vec_most_similar = np.argsort(vec_sim, axis=1)[:, -2]

類似文書を最初の10件表示してみましょう。

In [ ]:
for i, sim in enumerate(vec_most_similar):
    print("**************************************")
    print(data["text"][i])
    print("----")
    print(data["text"][sim])
    if i > 10:
        break